In [1]:

#DMF MicroData for MadhyaPradesh   
#Author: Niranjan 
#Advisor: Prof. Sabyasachi Das 

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.service import Service 
import time
import pandas as pd
import random 
import csv 
from pathlib import Path

import easyocr
import numpy as np
import cv2





C:\Users\niran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
option = webdriver.FirefoxOptions()
option.set_preference("browser.download.manager.showWhenStarting", False)
option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                       'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                       ';application/vnd.ms-excel'
                       ';application/xls'
                       ';application/xlsx')


In [3]:
# Open browser and hit the link

driver= webdriver.Firefox(options=option)

driver.get('https://mppanchayatdarpan.gov.in/PanchParmeshwar/Public/GetWorksListAll.aspx?DistId=MTQ=&LBId=MjQxMDg=&GPId=NjEyMA==&TypeId=MA==')




In [ ]:
#function to solve captcha 
def Captcha_Solve():
    import sys
    from twocaptcha import TwoCaptcha
    result = None
    
    sitekey = '<your_site_key_from_two_captcha>'
    api_key = '<your_api_key_from_two_captcha>'
    solver = TwoCaptcha(api_key)
    try:
        result = solver.recaptcha(
            sitekey=sitekey,
            url='https://www.deezer.com/en/login'
        )

    except Exception as e:
        sys.exit(e)

    return result

In [ ]:
## select the elements of dropdown menu 

##step-1. Select District 

select_district = driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_dddistrict"]')  #get the select element   

options = select_district.find_elements(By.TAG_NAME, "option") #get all the options into a list

#save into a list to get the name of the district 
Districts = [i.text for i in options]

#check if it is printing all the districts name correctly 
#print(optionsList)

#Now select each district and click in iteration 
for i in range(0, len(Districts), 1):

    ###print("starting loop on option %s" % i)
    
    print(Districts[i])
    #select the district  
    select_D = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_dddistrict"]'))

    ##condition to stop the loop 
    if i <53:
       select_D.select_by_index(i+1)

       time.sleep(1)

       #Step-2 select the localbody/block name 

       select_Local_Body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlb"]')))
       
       option_LB = select_Local_Body.find_elements(By.TAG_NAME, "option")

       LocalBody = [i.text for i in option_LB]

       print(LocalBody)

       for j in range(1, len(LocalBody)): 
           
           print(LocalBody[j])

           select_LB= Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlb"]'))
              
           #select a localbody 
           select_LB.select_by_index(j)


           ##Step-3 For Gram Panchayat 
           select_Gram_Panchayat = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlgz"]')))
       
           option_GP = select_Local_Body.find_elements(By.TAG_NAME, "option")

           #Save all GP under the block in a list 
           Gram_Panchayat = [i.text for i in option_GP]

           #print(Gram_Panchayat)

           for k in range(1, len(Gram_Panchayat)): 

               print(Gram_Panchayat[k])

               #select a Gram Panchayat and iterate over 

               select_GP = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlgz"]'))

               select_GP.select_by_index(k)

               
               #step-4. For Village Selection 

               select_village = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddvw"]')))
            
               option_VW = select_village.find_elements(By.TAG_NAME, "option")

               Village = [i.text for i in option_VW]

               #print(Village)
               
               for l in range(1, len(Village)): 
                   
                   print(Village[l])
                   
                   #select a Village 

                   select_VW = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddvw"]'))
                    
                   select_VW.select_by_index(l)

                   #select the scheme (static)
                   scheme = Select(driver.find_element(By.ID, 'ctl00_ctl00_ContentPlaceHolder1_cph_ddlScheme'))

                   scheme.select_by_visible_text('Pradhanmantri Khanij Kshetra Kalyan Yojna (PMKKKY)')
              
                   #scheme.select_by_index(64)  #or select by index 
                   

                   ##Solve Captcha 
                    
                   image = driver.find_element(By.CSS_SELECTOR, '.table > tbody:nth-child(1) > tr:nth-child(5) > td:nth-child(1) > div:nth-child(1) > span:nth-child(1) > img:nth-child(1)')
                   image.screenshot('captcha.png')
                   
                   ##convert the image into the readbale text format and save in string format 

                   reader = easyocr.Reader(['en'],gpu = False) # load once only in memory.

                   image_file_name= 'captcha.png' # this is the screen snap of your image

                   image = cv2.imread(image_file_name)

                   image_text = (reader.readtext(image, detail=0)[0])

                   print(image_text)
                 
                   ##input the captcha code 
                   driver.find_element(By.XPATH, '/html/body/form/div[4]/div[7]/table/tbody/tr[5]/td/div/span[2]/input').send_keys(image_text)
                           
                   ##click the search button 

                   try: 

                       submit_button = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[7]/div/a[1]/i').click()

                       time.sleep(4)

                   #handle the wrong captcha input 
                   a = submit_button.switch_to_alert

                   # accept the alert
                   a.accept()

                   driver.close
                   ##Check new page and condition it 

                   #Get all the urls available in the table 

                   project = driver.find_elements(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_gvWorks_ctl02_btnSelect"]')

                   project_urls = [i.get_attribute("href") for i in project]

                   if len(project_urls) == 0: 
                      
                      #go back to 
                      driver.back()

                   else: 
                   
                      for url in project_urls: 

                         #click on the url of the project 

                         print(url)
                          
                         driver.get(url)

                         time.sleep(1) 
                         #go back to previous project url 
                         driver.back()
                   
    else: 
        break 




SyntaxError: expected 'except' or 'finally' block (4259204649.py, line 128)